In [41]:
import pathlib
import itertools
import pickle

import pandas as pd
import geopandas as gpd
import shapely.ops
import shapely

import requests
import networkx as nx



%matplotlib inline

In [22]:
data_dir = pathlib.Path('~/data/drought').expanduser()


In [30]:
ivs_path = data_dir / 'ivs' / 'ivs2022_reizen.csv'
uneco_path = data_dir / 'unlocode' / 'unlocode.geojson'
graph_path = data_dir / 'network' / 'network_digital_twin_v0.3.pickle'

In [24]:
ivs_df = pd.read_csv(ivs_path)
ivs_df.head()

,jaar,week,v40_1_herkomst_land,v40_2_herkomst_plaats,v41_1_bestemming_land,v41_2_bestemming_plaats,lading,reizen,beladen_lvm
0,2022,1,NaN,NaN,NaN,NaN,NaN,23,NaN
1,2022,1,BE,AAB,NL,IJM,NaN,1,NaN
2,2022,1,BE,ANR,BE,ANR,555.0,1,4100.0
3,2022,1,BE,ANR,BE,AVL,NaN,1,NaN
4,2022,1,BE,ANR,BE,BGS,1200.0,1,1596.0


In [81]:
uneco_codes_in_ivs = set(ivs_df.groupby(['v40_1_herkomst_land', 'v40_2_herkomst_plaats']).first().index)

In [62]:
uneco_gdf = gpd.read_file(uneco_path)
uneco_columns = ['country_code', 'location_code', 'geometry']
# drop  1 of. Bruxelles / Brussel
uneco_gdf = uneco_df.drop_duplicates(subset=uneco_columns, keep='first')

In [86]:
in_ivs = uneco_gdf.apply(lambda row: (row['country_code'], row['location_code']) in uneco_codes_in_ivs, axis=1)
uneco_gdf['in_ivs'] = in_ivs

In [87]:
with graph_path.open('rb') as f:
    graph = pickle.load(f)

nodes_df = pd.DataFrame.from_dict(graph.nodes.values())
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry=nodes_df['geometry'].apply(shapely.geometry.shape), crs='EPSG:4326')

In [107]:
uneco_nodes_gdf = uneco_gdf.query('in_ivs').to_crs('EPSG:32631').sjoin_nearest(
    nodes_gdf.to_crs('EPSG:32631'), 
    how='left', 
    max_distance=2000
).to_crs('EPSG:4326')


In [108]:
uneco_nodes_gdf.to_file(data_dir / 'network' / 'uneco_nodes.geojson')

In [109]:
merged_gdf = (
    ivs_df
    .merge(
        uneco_nodes_gdf[uneco_columns + ['n']], 
        how='left',
        left_on=['v40_1_herkomst_land', 'v40_2_herkomst_plaats'], 
        right_on=['country_code', 'location_code'],
        suffixes=['', '_herkomst']
    )
    .merge(
        uneco_nodes_gdf[uneco_columns + ['n']], 
        how='left',
        left_on=['v41_1_bestemming_land', 'v41_2_bestemming_plaats'], 
        right_on=['country_code', 'location_code'],
        suffixes=['', '_bestemming']
    )    
)

In [110]:
merged_gdf

,jaar,week,v40_1_herkomst_land,v40_2_herkomst_plaats,v41_1_bestemming_land,v41_2_bestemming_plaats,lading,reizen,beladen_lvm,country_code,location_code,geometry,n,country_code_bestemming,location_code_bestemming,geometry_bestemming,n_bestemming
0,2022,1,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,NaN,None,NaN,NaN,NaN,None,NaN
1,2022,1,BE,AAB,NL,IJM,NaN,1,NaN,BE,AAB,POINT (4.03333 50.93333),L1311278_A,NL,IJM,POINT (4.60000 52.46667),B41489_A
2,2022,1,BE,ANR,BE,ANR,555.0,1,4100.0,BE,ANR,POINT (4.41667 51.21667),NaN,BE,ANR,POINT (4.41667 51.21667),NaN
3,2022,1,BE,ANR,BE,AVL,NaN,1,NaN,BE,ANR,POINT (4.41667 51.21667),NaN,BE,AVL,POINT (3.43333 50.76667),NaN
4,2022,1,BE,ANR,BE,BGS,1200.0,1,1596.0,BE,ANR,POINT (4.41667 51.21667),NaN,BE,BGS,POINT (3.23333 51.21667),FN143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116363,2022,39,SE,GOT,BE,GNE,23922.0,5,NaN,SE,GOT,POINT (11.96667 57.71667),NaN,BE,GNE,POINT (3.71667 51.05000),FN488
116364,2022,39,SE,UDD,NL,TNZ,NaN,1,NaN,SE,UDD,POINT (11.93333 58.36667),NaN,NL,TNZ,POINT (3.81667 51.46667),NaN
116365,2022,39,TR,DRC,NL,TNZ,31606.0,1,NaN,NaN,NaN,None,NaN,NL,TNZ,POINT (3.81667 51.46667),NaN
116366,2022,39,US,BTR,BE,GNE,52286.0,1,NaN,US,BTR,POINT (-91.13333 30.45000),NaN,BE,GNE,POINT (3.71667 51.05000),FN488


In [111]:
merged_gdf.to_pickle(data_dir / 'ivs' / 'ivs-geocoded.pickle')

In [104]:
# check if we only have 1 location for Brussel
uneco_df.query('location_code == "BRU" and country_code == "BE"')

,change,comment,country_code,date,function,iata,lat,latlon,location_code,lon,name,name_ascii,status,subdivision,geometry
4279,,,BE,1101,1234----,,50.833333,5050N 00420E,BRU,4.333333,Brussel (Bruxelles),Brussel (Bruxelles),AI,BRU,POINT (4.33333 50.83333)


In [105]:
(
    merged_gdf
    
    .groupby(['v41_1_bestemming_land', 'v41_2_bestemming_plaats'])
    .sum()
    .sort_values('reizen')
    .tail(n=10)
)



/var/folders/fh/tcbrjg6n28b0lzjzh07b5t6m0000gn/T/ipykernel_40107/2257658460.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  merged_gdf


jaar    week      lading  \
v41_1_bestemming_land v41_2_bestemming_plaats                                 
DE                    DUI                       1441686   14206   4408264.0   
NL                    MOE                       3251376   31445   2618324.0   
                      BON                       4246200   41394    861792.0   
                      VLI                       3307992   32995   2178654.0   
                      LEY                       2476950   25599    944616.0   
                      TNZ                       3206892   31595   4644035.0   
BE                    GNE                       7165968   70121  25808954.0   
                      ANR                       9938130   98088  21101472.0   
NL                    AMS                      12471696  125863  18426363.0   
                      RTM                      13729380  137811  22287075.0   

                                               reizen  beladen_lvm  
v41_1_bestemming_land v41_2_bestemming_plaats                       
DE                    DUI                        3343   10210447.0  
NL                    MOE                        3771    4286478.0  
                      BON                        4072    1666379.0  
                      VLI                        4439    3846009.0  
                      LEY                        4659    1635696.0  
                      TNZ                        5050    3058824.0  
BE                    GNE                       10999   13086825.0  
                      ANR                       21354   38195718.0  
NL                    AMS                       24380   28763933.0  
                      RTM                       33881   46067607.0

In [ ]:
# waypoints = {
#     ("NL", "RTM"): ["8865822", "8865822"], 
#     ("BE", "ANR"): ["8865822", "FN590"],
#     ("DE", "SGW"): ["8865822", "L1296169_A"],
#     ("NL", "AMS"): ["8865822", "8863932"],
#     ("DE", "DHU"): ["8865822", "FN584"], # Dortmund?
#     ("BE", "GNE"): ["8865822", "8868372", "35228510"],
#     ("DE", "DUI"): ["8865822", "FN96"],
#     ("DE", "WLM"): ["8865822", "FN0"], #Wesel
#     ("DE", "LUH"): ["8865822", "FN208"], # Ludwigshaven
# }
# rows = []
# for key, waypoints_i in waypoints.items():
#     row = {
#         "country": key[0],
#         "place": key[1],
#         "waypoints": waypoints_i
#     }
#     rows.append(row)
# rows

In [ ]:
waypoints = [
    {'country': 'BE', 'place': 'ANR', 'waypoints': ['8865822', 'FN590']},
    {'country': 'DE', 'place': 'SGW', 'waypoints': ['8865822', 'L1296169_A']},
    {'country': 'NL', 'place': 'AMS', 'waypoints': ['8865822', '8863932']},
    {'country': 'DE', 'place': 'DHU', 'waypoints': ['8865822', 'FN584']},
#     {'country': 'BE',
#     'place': 'GNE',
#     'waypoints': ['8865822', '8868372', '35228510']},
    {'country': 'DE', 'place': 'DUI', 'waypoints': ['8865822', 'FN96']},
    {'country': 'DE', 'place': 'WLM', 'waypoints': ['8865822', 'FN0']},
    {'country': 'DE', 'place': 'LUH', 'waypoints': ['8865822', 'FN208']}
]
waypoints

In [ ]:
def align_linestrings_from_end(df_i):
    for i, row in df_i[:-1].iterrows():
        geom = row['geometry']
        if geom.type == 'Point':
            continue
        next_row = df_i.loc[i+1]
        next_geom = next_row['geometry']
        if next_geom.type == 'Point':
            next_first_point = next_geom
        else:
            next_first_point = shapely.geometry.Point(next_geom.coords[0])
        last_point = shapely.geometry.Point(geom.coords[-1])
        first_point = shapely.geometry.Point(geom.coords[0])
        
        # are we in order
        in_order = last_point.distance(next_first_point) < first_point.distance(next_first_point)
        
        if not in_order:
            print('inverting', i)
            inverted_geom = shapely.geometry.LineString(geom.coords[::-1])
            geom = inverted_geom
            
        
    return df_i

In [ ]:
url = 'https://dtv-backend.azurewebsites.net/find_route'
legs = []
for row in waypoints:
    body = {
        "waypoints": row['waypoints']
    }
    print(body)
    resp = requests.post(url, json=body)
    
    row['route'] = resp.json()
    legs_df_i = gpd.GeoDataFrame.from_features(row['route']['features'])
    row['legs'] = align_linestrings_from_end(legs_df_i)
    del row['route']
    row['source_country'] = 'NL'
    row['source_city'] = 'AMS'
    row['target_country'] = row['country']
    row['target_city'] = row['place']
    legs.append(row)

In [ ]:
legs_df = pd.DataFrame(legs)

In [ ]:
trips_df = pd.merge(
   legs_df, 
    df,
    left_on=['source_country', 'source_city', 'target_country', 'target_city'],
    right_on=['v40_1_herkomst_land', 'v40_2_herkomst_plaats', 'v41_1_bestemming_land', 'v41_2_bestemming_plaats']
)
    

In [ ]:
legs_df_i = legs_df['legs'].iloc[0]

In [ ]:
legs_df_i['geometry'][0].coords[-1], legs_df_i['geometry'][1].coords[0]

In [ ]:
def linemerge(legs_df_i):
    merged = shapely.ops.linemerge([geom for geom in legs_df_i['geometry'] if geom.type == 'LineString'])
    if merged.type == 'MultiLineString':
        lines = []
        for line_a, line_b in zip(list(merged.geoms)[:-1], list(merged.geoms)[1:]):
            lines.append(line_a)
            end_a = line_a.coords[-1]
            start_b = line_b.coords[0]
            line_ab = shapely.geometry.LineString([end_a, start_b])
            lines.append(line_ab)
        # add last line
        lines.append(line_b)
        # merge again
        merged = shapely.ops.linemerge(lines)
        
    assert merged.type == 'LineString', merged.type
    return merged

trips_df['linestring'] = trips_df['legs'].apply(linemerge)

In [ ]:
processed_gdf = gpd.GeoDataFrame(trips_df.drop(columns=['legs', 'waypoints']), geometry='linestring')

In [ ]:
processed_gdf.to_file(data_dir / 'trips.geojson')

In [ ]:
processed_gdf.plot.scatter(x='reizen', y='lading')

In [ ]:
week_trip = processed_gdf.iloc[0]

In [ ]:
processed_gdf